In [1]:
import re
from sentence_transformers import util, SentenceTransformer
import time
import os
import json

In [21]:
import spacy

# Load the English tokenizer, tagger, parser, etc.
nlp = spacy.load("en_core_web_sm")

# 1. Set LLM & database
llm_name = 'kforth/IfcMaterial2MP'
llm = SentenceTransformer(llm_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/727k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/196 [00:00<?, ?B/s]

In [4]:
# 2. Load relevant BoM JSONS
# 2.1 Set relevant material-specific element layers
relevantBoM = ['Basic-Wall-Holzfassade-hinterlüftet-1105047_L1',
               'Floor-Dämmung-Galerie-1708478_L2',
               'Basic-Wall-Blechverkleidung-gedämmt-1829613_L2',
               'Basic-Wall-Holzbau_Archisonic_Vorbauschale-2146702_L1',
               'Basic-Wall-Holzbau_Archisonic_Vorbauschale-2146702_L2',
               'Basic-Wall-Holzfassade-hinterlüftet-1105047_L1']

In [3]:
# 2.2 Load related BoM dictionaries
bom_directory_path = '03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers'

In [17]:
import requests


url = "https://raw.githubusercontent.com/jakob-beetz/sbe-2025-lca-workshop/refs/heads/main/data/zip_files/03_BoM.zip"
output_path = "./03_BoM.zip"
response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)

print("✅ Bill of materials downloaded.")

✅ Bill of materials downloaded.


In [14]:
import zipfile

with zipfile.ZipFile('03_BoM.zip', 'r') as zip_ref:
    all_files = [f for f in zip_ref.namelist() if not f.endswith('/') and f.startswith(bom_directory_path)]
    print(all_files)


['03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers/Basic-Wall-Blechverkleidung-gedämmt-1829613_L2.json', '03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers/Basic-Wall-Holzbau_Archisonic_Vorbauschale-2146702_L1.json', '03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers/Basic-Wall-Holzbau_Archisonic_Vorbauschale-2146702_L2.json', '03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers/Basic-Wall-Holzfassade-hinterlüftet-1105047_L1.json', '03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers/Basic-Wall-Holzfassade-hinterlüftet-1105047_L2.json', '03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers/dummy.txt', '03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers/Floor-Dämmung-Galerie-1708478_L2.json', '03_BoM/samples_HiLo/HiLo_Samples_Processed/Include_Geometry_Data/Target_Layers/Floor-Funicular-Flo

In [15]:
relevantBoMdict = {}

with zipfile.ZipFile('03_BoM.zip') as z:
  for filename in all_files:
      basename = filename.split('/')[-1].split('.json')[0]
      if basename in relevantBoM:
          with z.open(filename) as f:
              data = json.load(f)
          relevantBoMdict[basename] = data

relevantBoMdict.keys()

dict_keys(['Basic-Wall-Blechverkleidung-gedämmt-1829613_L2', 'Basic-Wall-Holzbau_Archisonic_Vorbauschale-2146702_L1', 'Basic-Wall-Holzbau_Archisonic_Vorbauschale-2146702_L2', 'Basic-Wall-Holzfassade-hinterlüftet-1105047_L1', 'Floor-Dämmung-Galerie-1708478_L2'])

In [16]:
relevantBoMdict['Basic-Wall-Blechverkleidung-gedämmt-1829613_L2']['Target Layer of Material Inference']['Material Name']

'Wärmedämmung Slentex Aerogel'

In [39]:
material_names = [relevantBoMdict[layer_material]['Target Layer of Material Inference']['Material Name']
                  for layer_material in relevantBoMdict]
material_names

['Wärmedämmung Slentex Aerogel',
 'Archisonic Charcoal',
 'MDF',
 'Holzschalung Vertikal, Nadelholz,  schwarz gestrichen',
 'Wärmedämmung Mineralwolle, 0.035 W/mK']

In [40]:
material_name_embedings = llm.encode(material_names)
material_name_embedings.shape

(5, 768)

In [41]:
material_names_tokens = [[token.text for token in nlp(material_name) if not token.is_punct and not token.is_space] for material_name in material_names]
material_names_tokens

[['Wärmedämmung', 'Slentex', 'Aerogel'],
 ['Archisonic', 'Charcoal'],
 ['MDF'],
 ['Holzschalung', 'Vertikal', 'Nadelholz', 'schwarz', 'gestrichen'],
 ['Wärmedämmung', 'Mineralwolle', '0.035', 'W', 'mK']]

In [42]:
all_material_name_tokens = []
for material_name, material_name_tokens in zip(material_names, material_names_tokens):
    all_material_name_tokens.append([material_name] + material_name_tokens)

all_material_name_tokens

[['Wärmedämmung Slentex Aerogel', 'Wärmedämmung', 'Slentex', 'Aerogel'],
 ['Archisonic Charcoal', 'Archisonic', 'Charcoal'],
 ['MDF', 'MDF'],
 ['Holzschalung Vertikal, Nadelholz,  schwarz gestrichen',
  'Holzschalung',
  'Vertikal',
  'Nadelholz',
  'schwarz',
  'gestrichen'],
 ['Wärmedämmung Mineralwolle, 0.035 W/mK',
  'Wärmedämmung',
  'Mineralwolle',
  '0.035',
  'W',
  'mK']]

In [43]:
import pandas as pd

material_names_and_tokens_df = pd.DataFrame({
    'material_names': material_names,
    'material_name_tokens': all_material_name_tokens
})

material_names_and_tokens_df = material_names_and_tokens_df.explode('material_name_tokens')
material_names_and_tokens_df = material_names_and_tokens_df.groupby(['material_names'], as_index=True).apply(lambda x: x, include_groups=False).reset_index()
material_names_and_tokens_df

,material_names,level_1,material_name_tokens
0,Archisonic Charcoal,1,Archisonic Charcoal
1,Archisonic Charcoal,1,Archisonic
2,Archisonic Charcoal,1,Charcoal
3,"Holzschalung Vertikal, Nadelholz, schwarz ges...",3,"Holzschalung Vertikal, Nadelholz, schwarz ges..."
4,"Holzschalung Vertikal, Nadelholz, schwarz ges...",3,Holzschalung
5,"Holzschalung Vertikal, Nadelholz, schwarz ges...",3,Vertikal
6,"Holzschalung Vertikal, Nadelholz, schwarz ges...",3,Nadelholz
7,"Holzschalung Vertikal, Nadelholz, schwarz ges...",3,schwarz
8,"Holzschalung Vertikal, Nadelholz, schwarz ges...",3,gestrichen
9,MDF,2,MDF


In [44]:
url = "https://raw.githubusercontent.com/jakob-beetz/sbe-2025-lca-workshop/refs/heads/main/data/zip_files/01_OBD-database.zip"
output_path = "./01_OBD-database.zip"
response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)

print("✅ ÖKOBAUDAT dataset downloaded.")

✅ ÖKOBAUDAT dataset downloaded.


In [45]:
import zipfile
import pandas as pd
from collections import defaultdict


if os.path.isfile('obd_df.csv'):
    obd_df = pd.read_csv('obd_df.csv')
else:
    zip_path = "./01_OBD-database.zip"

    folder_data = set()
    leaf_file_data = []

    with zipfile.ZipFile(zip_path) as z:
        all_paths = z.namelist()

        # Split into folders and files
        folders = {p for p in all_paths if p.endswith('/')}
        files = [p for p in all_paths if not p.endswith('/')]

        # Map folders to their subfolders
        folder_children = defaultdict(set)
        for folder in folders:
            for other in folders:
                if other != folder and other.startswith(folder):
                    folder_children[folder].add(other)

        # Identify leaf folders (no subfolders)
        leaf_folders = {f for f in folders if not folder_children[f]}

        # Collect folder data
        for folder in folders:
            parts = folder.strip('/').split('/')
            name = parts[-1]
            parent = parts[-2] if len(parts) > 1 else ''
            depth = len(parts)
            folder_data.add((name, parent, depth, True))

        # Collect files in leaf folders
        for file_path in files:
            for folder in leaf_folders:
                if file_path.startswith(folder) and '/' not in file_path[len(folder):]:
                    parts = file_path.strip('/').split('/')
                    name = parts[-1]
                    if name != 'index.json': continue
                    with z.open(file_path) as f:
                            json_content = json.load(f)
                    for item in json_content["items"]:
                        if "Name" in item:
                            #print(item["Name"])
                            parent = parts[-2] if len(parts) > 1 else ''
                            depth = len(parts)
                            leaf_file_data.append((item["Name"], parent, depth, False))

    # Combine and create DataFrame
    all_data = list(folder_data) + leaf_file_data
    obd_df = pd.DataFrame(all_data, columns=["name", "parent", "depth", "is_category"])
    obd_df = obd_df.sort_values(by=["depth", "parent", "name", "is_category"]).reset_index(drop=True)
    obd_df = obd_df[obd_df['depth'] > 1].reset_index(drop=True)
    obd_df.to_csv('obd_df.csv', index=False)

obd_df[obd_df['parent'] == 'Dämmstoffe']


,name,parent,depth,is_category
17,Baumwolle,Dämmstoffe,3,True
18,Blähperlit,Dämmstoffe,3,True
19,Calciumsilikat,Dämmstoffe,3,True
20,Dämmelemente,Dämmstoffe,3,True
21,Expandierter_Kork,Dämmstoffe,3,True
22,Expandiertes_Polystyrol_(EPS),Dämmstoffe,3,True
23,Extrudiertes_Polystyrol_(XPS),Dämmstoffe,3,True
24,Flachsfaser,Dämmstoffe,3,True
25,Hanffaser,Dämmstoffe,3,True
26,Harnstoff-Formaldehydharz,Dämmstoffe,3,True


In [46]:
import numpy as np

url = f'https://raw.githubusercontent.com/jakob-beetz/sbe-2025-lca-workshop/refs/heads/main/data/zip_files/obd_embeddings_llm_{llm_name.split("/")[1]}.npy'
output_path = f'./obd_embeddings_llm_{llm_name.split("/")[1]}.npy'
response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)

obd_embeddings = np.load(f'obd_embeddings_llm_{llm_name.split("/")[1]}.npy')
print("✅ ÖKOBAUDAT embeddings downloaded.")

✅ ÖKOBAUDAT embeddings downloaded.


In [47]:
obd_embeddings.shape

(2264, 768)

In [48]:
similarity_matrix = llm.similarity(llm.encode(material_names_and_tokens_df['material_name_tokens']), obd_embeddings)
similarity_matrix.shape

torch.Size([21, 2264])

In [49]:
similarity_matrix_df = pd.DataFrame(similarity_matrix)

In [50]:
def find_best_match(group):
  for depth in range(obd_df['depth'].min(), obd_df['depth'].max()+1):
    group[f'material_match_step_{depth-1}'] = ''
    group[f'material_score_step_{depth-1}'] = ''
    group[f'best_material_match_step_{depth-1}'] = ''
    group[f'best_material_score_step_{depth-1}'] = ''

  current_group_indices = group.index
  group = group.reset_index()
  best_category_match = ''

  for depth in range(obd_df['depth'].min(), obd_df['depth'].max()+1):
    obd_filter = obd_df['depth'] == depth
    if best_category_match != '':
      obd_filter = ((obd_df['depth'] == depth) & (obd_df['parent'] == best_category_match))
    similarity_matrix_filtered = similarity_matrix_df.loc[current_group_indices, obd_filter]
    if similarity_matrix_filtered.shape[1] == 0:
      continue
    group[f'material_match_step_{depth-1}'] = obd_df['name'].iloc[similarity_matrix_filtered.idxmax(axis=1).values].reset_index(drop=True)
    group[f'material_score_step_{depth-1}'] = similarity_matrix_filtered.max(axis=1).reset_index(drop=True)

    best_material_score_index = group[f'material_score_step_{depth-1}'].idxmax()
    #return best_material_score_index
    best_category_match = group.loc[best_material_score_index][f'material_match_step_{depth-1}']
    best_category_score = group.loc[best_material_score_index][f'material_score_step_{depth-1}']
    group[f'best_material_match_step_{depth-1}'] = best_category_match
    group[f'best_material_score_step_{depth-1}'] = best_category_score


  return group

material_names_and_tokens_matched_df = material_names_and_tokens_df.groupby('level_1').apply(find_best_match, include_groups=False)
material_names_and_tokens_matched_df
material_names_and_tokens_matched_df[['material_names', 'best_material_match_step_1', 'best_material_match_step_2', 'best_material_match_step_3', 'best_material_match_step_4']]

material_names  \
level_1                                                        
0       0                       Wärmedämmung Slentex Aerogel   
        1                       Wärmedämmung Slentex Aerogel   
        2                       Wärmedämmung Slentex Aerogel   
        3                       Wärmedämmung Slentex Aerogel   
1       0                                Archisonic Charcoal   
        1                                Archisonic Charcoal   
        2                                Archisonic Charcoal   
2       0                                                MDF   
        1                                                MDF   
3       0  Holzschalung Vertikal, Nadelholz,  schwarz ges...   
        1  Holzschalung Vertikal, Nadelholz,  schwarz ges...   
        2  Holzschalung Vertikal, Nadelholz,  schwarz ges...   
        3  Holzschalung Vertikal, Nadelholz,  schwarz ges...   
        4  Holzschalung Vertikal, Nadelholz,  schwarz ges...   
        5  Holzschalung Vertikal, Nadelholz,  schwarz ges...   
4       0              Wärmedämmung Mineralwolle, 0.035 W/mK   
        1              Wärmedämmung Mineralwolle, 0.035 W/mK   
        2              Wärmedämmung Mineralwolle, 0.035 W/mK   
        3              Wärmedämmung Mineralwolle, 0.035 W/mK   
        4              Wärmedämmung Mineralwolle, 0.035 W/mK   
        5              Wärmedämmung Mineralwolle, 0.035 W/mK   

          best_material_match_step_1 best_material_match_step_2  \
level_1                                                           
0       0                 Dämmstoffe     Wärmedämmverbundsystem   
        1                 Dämmstoffe     Wärmedämmverbundsystem   
        2                 Dämmstoffe     Wärmedämmverbundsystem   
        3                 Dämmstoffe     Wärmedämmverbundsystem   
1       0                End_of_Life                  Generisch   
        1                End_of_Life                  Generisch   
        2                End_of_Life                  Generisch   
2       0                Kunststoffe                Dichtmassen   
        1                Kunststoffe                Dichtmassen   
3       0                       Holz                   Vollholz   
        1                       Holz                   Vollholz   
        2                       Holz                   Vollholz   
        3                       Holz                   Vollholz   
        4                       Holz                   Vollholz   
        5                       Holz                   Vollholz   
4       0                 Dämmstoffe               Mineralwolle   
        1                 Dämmstoffe               Mineralwolle   
        2                 Dämmstoffe               Mineralwolle   
        3                 Dämmstoffe               Mineralwolle   
        4                 Dämmstoffe               Mineralwolle   
        5                 Dämmstoffe               Mineralwolle   

          best_material_match_step_3  \
level_1                                
0       0     Wärmedämmverbundsystem   
        1     Wärmedämmverbundsystem   
        2     Wärmedämmverbundsystem   
        3     Wärmedämmverbundsystem   
1       0                  Bauschutt   
        1                  Bauschutt   
        2                  Bauschutt   
2       0                        PUR   
        1                        PUR   
3       0      Konstruktionsvollholz   
        1      Konstruktionsvollholz   
        2      Konstruktionsvollholz   
        3      Konstruktionsvollholz   
        4      Konstruktionsvollholz   
        5      Konstruktionsvollholz   
4       0               Mineralwolle   
        1               Mineralwolle   
        2               Mineralwolle   
        3               Mineralwolle   
        4               Mineralwolle   
        5               Mineralwolle   

                                  best_material_match_step_4  
level_1                                        

In [51]:
material_names_and_tokens_matched_df[['material_names', 'best_material_match_step_1', 'best_material_match_step_2', 'best_material_match_step_3', 'best_material_match_step_4']].groupby('material_names').first()

,best_material_match_step_1,best_material_match_step_2,best_material_match_step_3,best_material_match_step_4
material_names,,,,
Archisonic Charcoal,End_of_Life,Generisch,Bauschutt,Construction rubble landfill
"Holzschalung Vertikal, Nadelholz, schwarz gestrichen",Holz,Vollholz,Konstruktionsvollholz,Cross-laminated timber
MDF,Kunststoffe,Dichtmassen,PUR,PUR sealing compound
"Wärmedämmung Mineralwolle, 0.035 W/mK",Dämmstoffe,Mineralwolle,Mineralwolle,Mineral wool (blowable)
Wärmedämmung Slentex Aerogel,Dämmstoffe,Wärmedämmverbundsystem,Wärmedämmverbundsystem,External thermal insulation composite system E...
